In [85]:
from sklearn.metrics import accuracy_score
from scipy.special import softmax
import alpaca.calibrator as calibrator
import numpy as np
import pandas as pd
from alpaca.dataloader.builder import build_dataset

In [86]:
import sklearn
import math
from torch.nn import functional as f
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader

In [87]:
def compute_errors(n_bins, probs, labels, len_dataset, threshold):
    ece = calibrator.compute_ece(n_bins, probs, labels, len_dataset)
    sce = calibrator.compute_sce(n_bins, probs, labels)
    ace = calibrator.compute_ace(n_bins, labels, probs)
    tace = calibrator.compute_tace(threshold, labels, probs, n_bins)
    errors = {
        'ece' : ece,
        'sce' : sce,
        'ace' : ace,
        'tace' : tace
    }
    for error in errors.items():
        print(str(error[0]), ' = ', error[1])

In [74]:
mnist = build_dataset('mnist', val_size=10_000)

In [75]:
X_train, y_train = mnist.dataset('train')
X_val, y_val = mnist.dataset('val')
X_cal = X_train[48000:][:]
X_train = X_train[0:48000][:]
y_cal = y_train[48000:][:]
y_train = y_train[0:48000][:]

x_shape = (-1, 1, 28, 28)

train_ds = TensorDataset(torch.FloatTensor(X_train.reshape(x_shape)), torch.LongTensor(y_train))
val_ds = TensorDataset(torch.FloatTensor(X_val.reshape(x_shape)), torch.LongTensor(y_val))
train_loader = DataLoader(train_ds, batch_size=512)
val_loader = DataLoader(val_ds, batch_size=512)
cal_ds = TensorDataset(torch.FloatTensor(X_cal.reshape(x_shape)), torch.LongTensor(y_cal))
cal_loader = DataLoader(cal_ds, batch_size=512)
X_val.shape

(10000, 784)

In [76]:
class Net(nn.Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = nn.Sequential(
            nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layers = nn.Sequential(
            nn.Linear(4 * 7 * 7, 10)
        )
  
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [77]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [78]:
for epoch in range(5):
    for x_batch, y_batch in train_loader: # Train for one epoch
        print('.', end='')
        prediction = model(x_batch)
        optimizer.zero_grad()
        loss = criterion(prediction, y_batch)
        loss.backward()
        optimizer.step()
    print('\nTrain loss on last batch', loss.item())

# Check accuracy
x_batch, y_batch = next(iter(val_loader))


class_preds = f.softmax(model(x_batch), dim=-1).detach().numpy()
predictions = np.argmax(class_preds, axis=-1)
print('Accuracy', accuracy_score(predictions, y_batch))

..............................................................................................
Train loss on last batch 0.5379677414894104
..............................................................................................
Train loss on last batch 0.3088064193725586
..............................................................................................
Train loss on last batch 0.22927790880203247
..............................................................................................
Train loss on last batch 0.18548454344272614
..............................................................................................
Train loss on last batch 0.15679685771465302
Accuracy 0.962890625


In [88]:
logits_list = []
labels_list = []
for x_batch, y_batch in cal_loader:
    logits_list.append(model(x_batch))
    labels_list.append(y_batch)
logits = torch.cat(logits_list)
labels = torch.cat(labels_list)
logits.detach_()


tensor([[-6.1167, -8.9871,  3.8799,  ..., -7.2181,  7.8800, -6.4360],
        [-0.1971,  4.4322, -0.6775,  ..., -2.8865,  2.0842, -4.1036],
        [-3.9227, -3.9793, -0.0458,  ..., -0.7269, -1.2891, -0.3848],
        ...,
        [-4.0691, -1.3082, -3.1546,  ..., -0.4103, -2.4649,  1.1635],
        [10.0700, -8.9821, -1.3832,  ..., -5.1656,  2.8142,  0.3769],
        [-7.6608, -3.4758, -6.1055,  ..., -0.1984,  0.0988,  6.5401]])

In [89]:
calibr = calibrator.ModelWithTempScaling(model, logits, labels)

In [90]:
calibr.scaling()

ModelWithTempScaling(
  (model): Net(
    (cnn_layers): Sequential(
      (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU(inplace=True)
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (linear_layers): Sequential(
      (0): Linear(in_features=196, out_features=10, bias=True)
    )
  )
)

In [91]:
val_logits_list = []
val_labels_list = []
for x_batch, y_batch in val_loader:
    val_logits_list.append(model(x_batch))
    val_labels_list.append(y_batch)
val_logits = torch.cat(val_logits_list)
val_labels = torch.cat(val_labels_list)
val_logits.detach_()

tensor([[-5.7242, -2.0955, -2.2245,  ...,  9.8173, -0.7770,  0.8910],
        [ 3.1985, -2.7507,  0.3069,  ..., -4.4638, -1.2120, -2.2296],
        [-2.6328, -4.6969,  7.7340,  ..., -3.8817, -2.4001, -3.3600],
        ...,
        [-0.9659, -0.9615, -2.7335,  ..., -0.5133,  1.3431, -1.5266],
        [-2.0694, -3.1440,  3.9460,  ...,  0.5145,  0.2711, -1.3649],
        [-0.1192, -1.5945,  0.3817,  ..., -4.0632, -0.5899, -1.9224]])

In [93]:
probs = f.softmax(val_logits, dim=-1)

In [94]:
compute_errors(n_bins=15, probs=probs.numpy(), labels=val_labels.numpy(),
               len_dataset=np.shape(probs)[0], threshold=0.9)

ece  =  tensor([0.0296])
sce  =  tensor([0.0033])
ace  =  tensor(0.0310)
tace  =  tensor(0.0147)


In [95]:
print(calibr.temperature)

Parameter containing:
tensor([0.6217], requires_grad=True)


In [96]:
temp_scaling_logits = torch.true_divide(val_logits, calibr.temperature)
temp_scaling_probs = f.softmax(temp_scaling_logits, dim=1)

In [97]:
compute_errors(n_bins=15, probs=temp_scaling_probs.detach().numpy(), labels=val_labels.numpy(),
               len_dataset=np.shape(probs)[0], threshold=0.9)

ece  =  tensor([0.0085])
sce  =  tensor([0.0015])
ace  =  tensor(0.0182)
tace  =  tensor(0.0070)


In [98]:
calibr = calibrator.ModelWithVectScaling(model, logits, labels).float()
labels.dtype

torch.int64

In [99]:
calibr.scaling(lr=0.01, max_iter=50)

ModelWithVectScaling(
  (model): Net(
    (cnn_layers): Sequential(
      (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU(inplace=True)
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (linear_layers): Sequential(
      (0): Linear(in_features=196, out_features=10, bias=True)
    )
  )
)

In [100]:
vect_scaling_logits = calibr.scaling_logits(val_logits)
vect_scaling_probs = f.softmax(vect_scaling_logits, dim=1)

In [101]:
compute_errors(n_bins=15, probs=vect_scaling_probs.detach().numpy(), labels=val_labels.numpy(),
               len_dataset=np.shape(probs)[0], threshold=0.9)

ece  =  tensor([0.0183])
sce  =  tensor([0.0024])
ace  =  tensor(0.0224)
tace  =  tensor(0.0110)


In [102]:
calibr.W_and_b

Parameter containing:
tensor([ 1.0942e+00,  1.0579e+00,  1.0807e+00,  1.0858e+00,  1.1143e+00,
         1.0998e+00,  1.0911e+00,  1.1073e+00,  1.0508e+00,  1.1783e+00,
        -9.8967e-04,  1.2546e-03, -7.5154e-03,  7.0824e-03,  2.1402e-03,
        -1.2270e-02, -2.2263e-03,  9.1528e-03, -1.4546e-02,  1.7918e-02],
       requires_grad=True)

In [103]:
calibr = calibrator.ModelWithMatrScaling(model, logits, labels).float()
calibr.scaling(lr=0.0001, max_iter=1000)

ModelWithMatrScaling(
  (model): Net(
    (cnn_layers): Sequential(
      (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU(inplace=True)
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (linear_layers): Sequential(
      (0): Linear(in_features=196, out_features=10, bias=True)
    )
  )
)

In [104]:
matr_scaling_logits = calibr.scaling_logits(val_logits)
matr_scaling_probs = f.softmax(matr_scaling_logits, dim=1)

In [105]:
compute_errors(n_bins=15, probs=matr_scaling_probs.detach().numpy(), labels=val_labels.numpy(),
               len_dataset=np.shape(probs)[0], threshold=0.9)

ece  =  tensor([0.0051])
sce  =  tensor([0.0014])
ace  =  tensor(0.0167)
tace  =  tensor(0.0061)


In [106]:
hist_binning_probs = calibrator.multiclass_histogram_binning(15, logits.numpy(), labels.numpy(), val_logits)

In [108]:
compute_errors(n_bins=15, probs=hist_binning_probs, labels=val_labels.numpy(),
               len_dataset=np.shape(probs)[0], threshold=0.9)

ece  =  tensor([0.0069])
sce  =  tensor([0.0016])
ace  =  tensor(0.0177, dtype=torch.float64)
tace  =  tensor(0.0129, dtype=torch.float64)
